In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import pandas as pd

# ==== SETUP ====
IMG_DIR = r"C:\Users\Timothy\OneDrive\Desktop\jpeg"
IMAGE_SIZE = 256
OUTPUT_CSV = "cancer_predictions.csv"
MODEL_PATH = "breast_cancer_model.h5"
# ===============

# Load pre-trained classifier (you must train this first, see below)
model = load_model(MODEL_PATH)

# Load all image file names
image_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

results = []

for fname in image_files:
    fpath = os.path.join(IMG_DIR, fname)
    
    # Load and preprocess image
    img = image.load_img(fpath, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    # Predict
    pred = model.predict(img_array)[0][0]
    label = "malignant" if pred > 0.5 else "benign"

    results.append((fname, label))
    print(f"{fname} → {label} ({pred:.2f})")

# Save to CSV
df = pd.DataFrame(results, columns=["filename", "label"])
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Labels saved to: {OUTPUT_CSV}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'breast_cancer_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Set up directories for benign and malignant images
train_dir = 'path_to_training_data'
val_dir = 'path_to_validation_data'

# Image data generators for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images
    rotation_range=30,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resizing images to fit the model
    batch_size=32,
    class_mode='binary'  # Binary classification (benign or malignant)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[checkpoint]
)

# Save the final model
model.save('final_model.h5')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_training_data'